In [36]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
from sklearn.neighbors import NearestNeighbors
from torchvision.models import ResNet18_Weights
import os
import time
from PIL import Image
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.transforms import functional as F


print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return the name of your GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

True
NVIDIA GeForce RTX 4050 Laptop GPU
Using device: cuda


In [9]:
# Set the path to your dataset folder (adjust as needed)
train_data_path = r'C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train'
train_image_paths = []
# # List all files and directories in the specified folder
for root, folder, files in os.walk(train_data_path):
    print(root)
    for file in files:
        if file.endswith(('.jpg', '.jpeg')):  # Adjust extensions as needed
            train_image_paths.append(os.path.join(root, file))

C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train
C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train\KLT_120
C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train\KLT_139
C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train\KLT_155
C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train\KLT_18
C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\train\KLT_182
C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_

In [2]:
# Set the path to your dataset folder (adjust as needed)
test_data_path = r'C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\test'
test_image_paths = []
# # List all files and directories in the specified folder
for root, folder, files in os.walk(test_data_path):
    for file in files:
        if file.endswith(('.jpg', '.jpeg')):  # Adjust extensions as needed
            test_image_paths.append(os.path.join(root, file))

In [3]:
val_data_path = r'C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset\\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\val'
val_image_paths = []
# # List all files and directories in the specified folder
for root, folder, files in os.walk(val_data_path):
    for file in files:
        if file.endswith(('.jpg', '.jpeg')):  # Adjust extensions as needed
            val_image_paths.append(os.path.join(root, file))

In [5]:
## 'C:\\Users\\preet\\.cache\\huggingface\\hub\\datasets--vincentw1997--public_SLC_Dataset
# \\snapshots\\bd1a3938524ae591e9e38cda04600057f51a1674\\AD\\test\\anomaly
# \\1003_20230508-163349_18_NOK_Cam4.jpg'

#['1003','20230508-163349','18','NOK','Cam4','1003_20230508-163349_18_NOK_Cam4']
#['image num','timestamp','conatiner type', 'defect label', 'camera angle', 'filename']
def extractFileMetadata(filepath) :
    file_name = filepath[filepath.rindex("\\") + 1 : filepath.rindex(".")]
    metadata = file_name.split('_')
    metadata.append(file_name)
    return metadata

def preserve_metadata_order(batch):
    """
    Custom collate function to handle metadata as a list of lists.
    Args:
        batch: List of tuples [(image1, metadata1), (image2, metadata2), ...]
    Returns:
        images: Stacked images (Tensor)
        metadata: List of metadata lists
    """
    images, metadata = zip(*batch)  # Unpack images and metadata
    return torch.stack(images, dim=0), list(metadata)

In [6]:

    
class CustomDataset(Dataset):
    def __init__(self, file_path, transform=None):
        if isinstance(file_path, str):
            self.file_path = [file_path]
        else:
            self.file_path = file_path
        self.transform = transform

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, i):
        file_path = self.file_path[i]
        image = Image.open(file_path).convert("RGB")  # Assuming "image" is the key for image data
        metadata = extractFileMetadata(file_path)

        if self.transform:
            image = self.transform(image)
        
        return image, metadata

In [7]:

def dynamic_adjustments(image):
    img_np = np.array(image.convert("L"))
    mean_intensity = img_np.mean()

    if mean_intensity < 100:
        image = F.adjust_brightness(image, 1.5)
    elif mean_intensity > 200:
        image = F.adjust_brightness(image, 0.8)

    contrast_factor = 1 + (128 - img_np.std()) / 128
    image = F.adjust_contrast(image, contrast_factor)
    return image

transform = transforms.Compose([
    transforms.Lambda(dynamic_adjustments),  # Dynamic adjustments
    transforms.Resize((224, 224)),          # Resize for ResNet18
    transforms.ToTensor(),                  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ImageNet
])


Define Dataset and Dataloaders

In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import numpy as np

# Create train and test datasets using Subset
train_dataset = CustomDataset(train_image_paths, transform=transform)
test_dataset = CustomDataset(test_image_paths, transform=transform)
val_dataset = CustomDataset(val_image_paths, transform=transform)


# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=preserve_metadata_order)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=preserve_metadata_order)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=preserve_metadata_order)


In [12]:
# Specify layers for feature extraction
model_layers = {
    "layer2": "features_layer2",
    "layer3": "features_layer3"
}

# Load pretrained ResNet18 model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Create a feature extractor
pretrained_model = create_feature_extractor(model, model_layers)
pretrained_model = pretrained_model.to(device)

In [10]:
unique_labels = torch.unique(torch.tensor([int(data[2]) for _, metadata in train_loader for data in metadata]))

label_mapping = {label.item(): idx for idx, label in enumerate(unique_labels)}
num_classes = len(unique_labels)  # Update the number of classes # Define based on your dataset
print(num_classes)

18


In [30]:
# Create a dictionary to map integer keys to sequential values
label_mapping = {
    18: 0,
    23: 1,
    24: 2,
    25: 3,
    31: 4,
    43: 5,
    120: 6,
    139: 7,
    155: 8,
    182: 9,
    194: 10,
    195: 11,
    200: 12,
    203: 13,
    206: 14,
    209: 15,
    215: 16,
    289: 17
}

In [20]:
# Define MultiTaskModel with Dropout for Regularization
class CADModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CADModel, self).__init__()
        self.shared_fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5)  # Add dropout
        )
        self.classification_fc = nn.Linear(512, num_classes)
        self.anomaly_fc = nn.Linear(512, 2)

    def forward(self, x):
        shared_features = self.shared_fc(x)
        class_output = self.classification_fc(shared_features)
        anomaly_output = self.anomaly_fc(shared_features)
        return class_output, anomaly_output

### Training loop

In [ ]:

# Initialize MultiTaskModel
new_tensor = torch.randn(1, 3, 224, 224).to(device)
features = pretrained_model(new_tensor)
feature_size = sum([features[layer].shape[1] * features[layer].shape[2] * features[layer].shape[3] for layer in model_layers.values()])
multi_task_model = CADModel(feature_size, num_classes).to(device)

# Freeze all layers except layer3 in the pretrained model
for name, param in pretrained_model.named_parameters():
    if "layer3" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Loss Functions and Optimizer with L2 Regularization
classification_loss_fn = nn.CrossEntropyLoss()
anomaly_loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    list(filter(lambda p: p.requires_grad, pretrained_model.parameters())) + 
    list(multi_task_model.parameters()),
    lr=0.001,
    weight_decay=1e-3  # Add weight decay
)

# Training Loop with Training Accuracy Calculation
num_epochs = 5
pretrained_model.train()
multi_task_model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0.0
    correct_classifications = 0
    correct_anomalies = 0
    total_samples = 0

    for images, metadata_batch in train_loader:
        images = images.to(device)
        container_labels = torch.tensor(
            [label_mapping[int(data[2])] for data in metadata_batch]
        ).to(device)
        anomaly_labels = torch.tensor(
            [0 if data[3].lower() == "ok" else 1 for data in metadata_batch]
        ).to(device)

        # Forward pass through pretrained model
        features = pretrained_model(images)

        # Extract and concatenate features
        combined_features = torch.cat(
            [features[layer].view(features[layer].size(0), -1) for layer in model_layers.values()],
            dim=1
        )

        # Forward pass through multi-task model
        class_outputs, anomaly_outputs = multi_task_model(combined_features)

        # Compute losses
        classification_loss = classification_loss_fn(class_outputs, container_labels)
        anomaly_loss = anomaly_loss_fn(anomaly_outputs, anomaly_labels)
        total_loss = classification_loss + anomaly_loss

        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Update metrics
        epoch_loss += total_loss.item()
        predicted_classes = torch.argmax(class_outputs, dim=1)
        predicted_anomalies = torch.argmax(anomaly_outputs, dim=1)
        correct_classifications += (predicted_classes == container_labels).sum().item()
        correct_anomalies += (predicted_anomalies == anomaly_labels).sum().item()
        total_samples += len(container_labels)

    # Calculate accuracies
    classification_accuracy = (correct_classifications / total_samples) * 100
    anomaly_accuracy = (correct_anomalies / total_samples) * 100

    print(f"Epoch {epoch + 1}/{num_epochs}, Total Loss: {epoch_loss:.4f}")
    print(f"Training Classification Accuracy: {classification_accuracy:.2f}%")
    print(f"Training Anomaly Detection Accuracy: {anomaly_accuracy:.2f}%")


Epoch 1/5
Epoch 1/5, Total Loss: 312.7547
Training Classification Accuracy: 86.09%
Training Anomaly Detection Accuracy: 78.70%
Epoch 2/5
Epoch 2/5, Total Loss: 102.1788
Training Classification Accuracy: 93.07%
Training Anomaly Detection Accuracy: 84.99%
Epoch 3/5
Epoch 3/5, Total Loss: 97.6373
Training Classification Accuracy: 93.60%
Training Anomaly Detection Accuracy: 86.67%
Epoch 4/5
Epoch 4/5, Total Loss: 93.0897
Training Classification Accuracy: 93.83%
Training Anomaly Detection Accuracy: 88.95%
Epoch 5/5
Epoch 5/5, Total Loss: 106.5404
Training Classification Accuracy: 91.81%
Training Anomaly Detection Accuracy: 88.13%


### Load the trained Model

In [27]:
# Load the checkpoint
load_models = torch.load("Finetuned_model.pth", map_location=device)


fine_tuned_model = CADModel(load_models['feature_size'], load_models['num_classes']).to(device)

# Load the state dictionaries
pretrained_model.load_state_dict(load_models['pretrained_model_state_dict'])
fine_tuned_model.load_state_dict(load_models['multi_task_model_state_dict'])

# Set models to evaluation mode
pretrained_model.eval()
fine_tuned_model.eval()


C:\Users\preet\AppData\Local\Temp\ipykernel_21792\1416893726.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_models = torch.load("Finetuned_model.pth", map_location

CADModel(
  (shared_fc): Sequential(
    (0): Linear(in_features=150528, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (classification_fc): Linear(in_features=512, out_features=18, bias=True)
  (anomaly_fc): Linear(in_features=512, out_features=2, bias=True)
)

### Testing/Validation loop and computing loss calculation

In [41]:
# Validation Loop
pretrained_model.eval()
multi_task_model.eval()
val_loss = 0.0
inference_times = []
correct_classifications = 0
correct_anomalies = 0
total_samples = 0

all_true_class_labels = []
all_predicted_class_labels = []
all_true_anomaly_labels = []
all_predicted_anomaly_labels = []
# Loss Functions and Optimizer with L2 Regularization
classification_loss_fn = nn.CrossEntropyLoss()
anomaly_loss_fn = nn.CrossEntropyLoss()
with torch.no_grad():
    for images, metadata_batch in test_loader:
        # Move data to device
        images = images.to(device)
        container_labels = torch.tensor(
            [label_mapping[int(data[2])] for data in metadata_batch]
        ).to(device)
        anomaly_labels = torch.tensor(
            [0 if data[3].lower() == "ok" else 1 for data in metadata_batch]
        ).to(device)

        # Start measuring time
        start_time = time.perf_counter()

        # Forward pass through the pretrained model
        features = pretrained_model(images)

        # Extract and concatenate features
        combined_features = torch.cat(
            [features[layer].view(features[layer].size(0), -1) for layer in model_layers.values()],
            dim=1
        )

        # Forward pass through the multi-task model
        class_outputs, anomaly_outputs = fine_tuned_model(combined_features)

        # End measuring time
        end_time = time.perf_counter()
        inference_times.append(end_time - start_time)

        # Calculate predictions
        predicted_classes = torch.argmax(class_outputs, dim=1)
        predicted_anomalies = torch.argmax(anomaly_outputs, dim=1)

        # Collect predictions and labels for classification report and confusion matrix
        all_true_class_labels.extend(container_labels.cpu().numpy())
        all_predicted_class_labels.extend(predicted_classes.cpu().numpy())
        all_true_anomaly_labels.extend(anomaly_labels.cpu().numpy())
        all_predicted_anomaly_labels.extend(predicted_anomalies.cpu().numpy())

        # Update accuracy metrics
        correct_classifications += (predicted_classes == container_labels).sum().item()
        correct_anomalies += (predicted_anomalies == anomaly_labels).sum().item()
        total_samples += len(container_labels)

        # Print predictions and ground truth
        print(f'Classification True: {container_labels}, Predicted: {predicted_classes}')
        print(f'Anomaly True: {anomaly_labels}, Predicted: {predicted_anomalies}')

        # Compute losses
        classification_loss = classification_loss_fn(class_outputs, container_labels)
        anomaly_loss = anomaly_loss_fn(anomaly_outputs, anomaly_labels)
        print(f"Classification Loss: {classification_loss:.4f}, Anomaly Loss: {anomaly_loss:.4f}")

        # Accumulate validation loss
        val_loss += (classification_loss + anomaly_loss).item()

    # Calculate average inference time
    avg_inference_time_per_batch = sum(inference_times) / len(inference_times)
    avg_inference_time_per_image = avg_inference_time_per_batch / len(images)
    


    # Calculate accuracies
    classification_accuracy = (correct_classifications / total_samples) * 100
    anomaly_accuracy = (correct_anomalies / total_samples) * 100

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Classification Accuracy: {classification_accuracy:.2f}%")
    print(f"Anomaly Detection Accuracy: {anomaly_accuracy:.2f}%")
    print(f"Average Inference Time per Batch: {avg_inference_time_per_batch:.6f} seconds")
    print(f"Average Inference Time per Image: {avg_inference_time_per_image:.6f} seconds")


Classification True: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0'), Predicted: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
Anomaly True: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), Predicted: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Classification Loss: 0.0001, Anomaly Loss: 0.0317
Classification True: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0'), Predicted: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
Anomaly True: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Testing

In [45]:
sample_test_data = "C:\\Users\\preet\\Downloads\\777_20230508-155957_289_OK_Cam3.jpg"  # either single image path or list of image path can be given like ["C:\\Users\\preet\\Downloads\\777_20230508-155957_289_OK_Cam3.jpg","C:\\Users\\preet\\Downloads\\777_20230508-155957_289_OK_Cam3.jpg"]
                    

testing_dataset = CustomDataset(sample_test_data, transform=transform)
testing_loader = DataLoader(testing_dataset,batch_size=1, shuffle=False, collate_fn=preserve_metadata_order)


inference_times = []

with torch.no_grad():
    for images, metadata_batch in testing_loader:
        # Move data to device
        images = images.to(device)
        print(metadata_batch)
   
        start_time = time.perf_counter()

        # Forward pass through the pretrained model
        features = pretrained_model(images)
        # Extract and concatenate features
        combined_features = torch.cat(
            [features[layer].view(features[layer].size(0), -1) for layer in model_layers.values()],
            dim=1
        )
        # Forward pass through the multi-task model
        class_outputs, anomaly_outputs = fine_tuned_model(combined_features)

        end_time = time.perf_counter()
        inference_time = end_time - start_time

        # Calculate predictions
        predicted_classes = torch.argmax(class_outputs, dim=1)
        predicted_anomalies = torch.argmax(anomaly_outputs, dim=1)
       
        reverse_label_mapping = {v: k for k, v in label_mapping.items()}
        decoded_predicted_class_labels = reverse_label_mapping.get(int(predicted_classes[0]))
        
        decoded_predicted_anomaly_labels = ['OK' if predicted_anomalies == 0 else 'NOK']
        print(f'decoded_predicted_class_labels : {decoded_predicted_class_labels} and decoded_predicted_anomaly_labels :  {decoded_predicted_anomaly_labels[0]}')
        print(f'time taken :  {inference_time*1000:.6f} ms')



[['777', '20230508-155957', '289', 'OK', 'Cam3', '777_20230508-155957_289_OK_Cam3']]
decoded_predicted_class_labels : 289 and decoded_predicted_anomaly_labels :  OK
time taken :  3.110600 ms
